## Real-Time Prediction (with CoinGecko API)

In [1]:
import requests
import pandas as pd
import joblib
from datetime import datetime

# ------------------------------
# 1. Load trained pipeline
# ------------------------------
pipeline = joblib.load("trained_pipeline.pkl")
print("✅ Trained pipeline loaded successfully!")

# ------------------------------
# 2. Fetch live price data from CoinGecko
# ------------------------------
def fetch_live_data(coin_id="dogecoin", vs_currency="usd", days=30):
    """
    Fetch last N days of historical data for the given coin.
    (CoinGecko automatically gives hourly data if days <= 90)
    """
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {"vs_currency": vs_currency, "days": days}
    response = requests.get(url, params=params)

    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.text}")

    data = response.json()

    # Prices are [timestamp, price]
    prices = data["prices"]
    df = pd.DataFrame(prices, columns=["timestamp", "price"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df


df_live = fetch_live_data("dogecoin", "usd", days=30)
print("📥 Live data fetched. Latest price:", df_live['price'].iloc[-1])

# ------------------------------
# 3. Feature Engineering (same as training)
# ------------------------------
def create_features(df):
    df = df.copy()
    df["lag_1"] = df["price"].shift(1)
    df["lag_2"] = df["price"].shift(2)
    df["lag_3"] = df["price"].shift(3)
    df["return_1"] = df["price"].pct_change(1)
    df["return_3"] = df["price"].pct_change(3)
    df["rolling_mean_3"] = df["price"].rolling(window=3).mean()
    df["rolling_std_3"] = df["price"].rolling(window=3).std()
    df["rolling_mean_7"] = df["price"].rolling(window=7).mean()
    df["rolling_std_7"] = df["price"].rolling(window=7).std()
    df = df.dropna()
    return df

df_feat = create_features(df_live)

# ------------------------------
# 4. Make a prediction
# ------------------------------
def predict_next_price(df, pipeline):
    latest = df.iloc[-1:]   # take last row with all features
    pred_return = pipeline.predict(latest.drop(columns=["timestamp", "price"]))[0]
    
    last_price = latest["price"].iloc[0]
    predicted_price = last_price * (1 + pred_return)

    print("\n📊 Real-Time Prediction Summary")
    print("-" * 40)
    print(f"🕒 Timestamp          : {latest['timestamp'].iloc[0]}")
    print(f"💰 Last Observed Price: ${last_price:.4f}")
    print(f"📈 Predicted Return   : {pred_return*100:.2f}%")
    print(f"🔮 Predicted Next Price: ${predicted_price:.4f}")
    print("-" * 40)

    return {
        "timestamp": latest["timestamp"].iloc[0],
        "last_price": last_price,
        "predicted_return": pred_return,
        "predicted_price": predicted_price
    }

results = predict_next_price(df_feat, pipeline)


C:\Users\amban\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\amban\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


✅ Trained pipeline loaded successfully!
📥 Live data fetched. Latest price: 0.26814264039558566

📊 Real-Time Prediction Summary
----------------------------------------
🕒 Timestamp          : 2025-09-16 20:48:29
💰 Last Observed Price: $0.2681
📈 Predicted Return   : 26.80%
🔮 Predicted Next Price: $0.3400
----------------------------------------
